In [1]:
import numpy as np
import requests
import os

# Working with the Kirchoff matrix of the Gaussian Network Model

## Downloading any PDB file

Let's first of all store the PDB id we want to download and the path where the file needs to be located:

In [2]:
pdb_id = '4NKK'
download_path = "./"

And now let's define an auxiliary function to download pdb files to be stored locally in our machine.

In [3]:
def fetch_pdb(pdb_id, download_path="./"):
        url = 'http://files.rcsb.org/download/{}.pdb'.format(pdb_id)
        try:
            res = requests.get(url, allow_redirects=True)
        except:
            print("Could not fetch pdb from {}".format(url))
            return 
        
        file_path = os.path.join(download_path, pdb_id + ".pdb")
        with open(file_path, "wb") as f:
            f.write(res.content)
        print(f"The {pdb_id} file was downloaded succesfully!")

Time to try it!

In [4]:
fetch_pdb(pdb_id, download_path=download_path)

The 4NKK file was downloaded succesfully!


## Parsing the positions of the CA atoms

We are going take an approach not very short and efficiente to obtain the coordinates of the CA atoms. But let's do it this way just once:

In [19]:
file_path = os.path.join(download_path, pdb_id + ".pdb")

CA_coordinates = []

with open(file_path) as fff:
    for line in fff.readlines():
        if line.startswith('ATOM  ') and line.split()[2]=='CA' :
          
            x, y, z = line.split()[6:9]
            CA_coordinates.append([float(x), float(y), float(z)])

Ok, `CA_coordinates` is a list of lists... let's convert it to a numpy array:

In [32]:
CA_coordinates = np.array(CA_coordinates)

In [21]:
#Número de carbonos alpha
len(CA_coordinates)

99

## Definiendo la matriz de Kirchhoff

In [35]:
def kirchhoff_matrix(CA):
  A=np.zeros((len(CA),len(CA)))
  for i in range(len(A)):
    for j in range(len(A)):
      if i==j:
        continue
      d=np.sqrt(np.dot((CA[i]-CA[j]),(CA[i]-CA[j])))
      if d<9:
        A[i][j]=-1
        A[i][i]+=-1
  return A

In [38]:
kirchhoff_matrix(CA_coordinates)

array([[-3., -1., -1., ...,  0.,  0.,  0.],
       [-1., -3., -1., ...,  0.,  0.,  0.],
       [-1., -1., -5., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ..., -5., -1., -1.],
       [ 0.,  0.,  0., ..., -1., -3., -1.],
       [ 0.,  0.,  0., ..., -1., -1., -2.]])